## Before we start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Install

In [6]:
!pip install ipywidgets ipython opencv-python numpy supervision tqdm ultralytics gradio


##Importing Libraries


In [7]:
import gradio as gr
import shutil
import ipywidgets as widgets
from IPython.display import display, clear_output
import io
import base64
import cv2
import numpy as np
import supervision as sv
from tqdm.notebook import tqdm
from ultralytics import YOLO
from collections import defaultdict, deque
from ultralytics import solutions

In [ ]:
# Define the process_video function
def process_video(video):

    # Define the target path where the video will be saved
    source_path = video
    target_path = "/content/drive/MyDrive/processed_VIDEO.mp4"

    # Save the uploaded video to the target path
    shutil.copy(video, target_path)

    cap = cv2.VideoCapture(video)
    assert cap.isOpened(), "Error reading video file"
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

    video_writer = cv2.VideoWriter("speed_management.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))
    speed_region = [(20, 400), (1080, 404), (1080, 360), (20, 360)]
    speed_estimator = solutions.SpeedEstimator(model="yolo11n.pt", region=speed_region, show=True)

    # Video processing parameters
    CONFIDENCE_THRESHOLD = 0.3
    IOU_THRESHOLD = 0.5
    MODEL_NAME = "yolov8x.pt"
    MODEL_RESOLUTION = 1280

    SOURCE = np.array([
        [740, 120],  #a
        [1120, 120], #b
        [1750,1080], #c
        [0, 1080]  #d
    ])

    TARGET_WIDTH = 25
    TARGET_HEIGHT = 70

    TARGET = np.array([
        [0, 0],
        [TARGET_WIDTH - 1, 0],
        [TARGET_WIDTH - 1, TARGET_HEIGHT - 1],
        [0, TARGET_HEIGHT - 1],
    ])

    frame_generator = sv.get_video_frames_generator(source_path=target_path)
    frame_iterator = iter(frame_generator)
    frame = next(frame_iterator)

    annotated_frame = frame.copy()
    annotated_frame = sv.draw_polygon(scene=annotated_frame, polygon=SOURCE, color=sv.Color.RED, thickness=4)
    sv.plot_image(annotated_frame)

    class ViewTransformer:

        def __init__(self, source: np.ndarray, target: np.ndarray) -> None:
            source = source.astype(np.float32)
            target = target.astype(np.float32)
            self.m = cv2.getPerspectiveTransform(source, target)

        def transform_points(self, points: np.ndarray) -> np.ndarray:
            if points.size == 0:
                return points

            reshaped_points = points.reshape(-1, 1, 2).astype(np.float32)
            transformed_points = cv2.perspectiveTransform(reshaped_points, self.m)
            return transformed_points.reshape(-1, 2)

    view_transformer = ViewTransformer(source=SOURCE, target=TARGET)

    model = YOLO(MODEL_NAME)

    video_info = sv.VideoInfo.from_video_path(video_path=source_path)
    frame_generator = sv.get_video_frames_generator(source_path=source_path)

    # tracer initiation
    byte_track = sv.ByteTrack(
        frame_rate=video_info.fps,
    )

    # annotators configuration
    thickness = sv.calculate_optimal_line_thickness(
        resolution_wh=video_info.resolution_wh
    )
    text_scale = sv.calculate_optimal_text_scale(
        resolution_wh=video_info.resolution_wh
    )
    bounding_box_annotator = sv.BoundingBoxAnnotator(
        thickness=thickness
    )
    label_annotator = sv.LabelAnnotator(
        text_scale=text_scale,
        text_thickness=thickness,
        text_position=sv.Position.BOTTOM_CENTER
    )
    trace_annotator = sv.TraceAnnotator(
        thickness=thickness,
        trace_length=video_info.fps * 2,
        position=sv.Position.BOTTOM_CENTER
    )

    polygon_zone = sv.PolygonZone(polygon=SOURCE)

    coordinates = defaultdict(lambda: deque(maxlen=video_info.fps))

    # open target video
    with sv.VideoSink(target_path, video_info) as sink:

        # loop over source video frame
        for frame in tqdm(frame_generator, total=video_info.total_frames):

            result = model(frame, imgsz=MODEL_RESOLUTION, verbose=False)[0]
            detections = sv.Detections.from_ultralytics(result)

            # filter out detections by class and confidence
            detections = detections[detections.confidence > CONFIDENCE_THRESHOLD]
            detections = detections[detections.class_id != 0]

            # filter out detections outside the zone
            detections = detections[polygon_zone.trigger(detections)]

            # refine detections using non-max suppression
            detections = detections.with_nms(IOU_THRESHOLD)

            # pass detection through the tracker
            detections = byte_track.update_with_detections(detections=detections)

            points = detections.get_anchors_coordinates(
                anchor=sv.Position.BOTTOM_CENTER
            )

            # calculate the detections position inside the target RoI
            points = view_transformer.transform_points(points=points).astype(int)

            # store detections position
            for tracker_id, [_, y] in zip(detections.tracker_id, points):
                coordinates[tracker_id].append(y)

            # format labels
            labels = []

            for tracker_id in detections.tracker_id:
                if len(coordinates[tracker_id]) < video_info.fps / 2:
                    labels.append(f"#{tracker_id}")
                else:
                    # calculate speed
                    coordinate_start = coordinates[tracker_id][-1]
                    coordinate_end = coordinates[tracker_id][0]
                    distance = abs(coordinate_start - coordinate_end)
                    time = len(coordinates[tracker_id]) / video_info.fps
                    speed = distance / time * 3.6
                    labels.append(f"#{tracker_id} {int(speed)} km/h")

            # annotate frame
            annotated_frame = frame.copy()
            annotated_frame = trace_annotator.annotate(
                scene=annotated_frame, detections=detections
            )
            annotated_frame = bounding_box_annotator.annotate(
                scene=annotated_frame, detections=detections
            )
            annotated_frame = label_annotator.annotate(
                scene=annotated_frame, detections=detections, labels=labels
            )

            # add frame to target video
            sink.write_frame(annotated_frame)


    return f"Video processed and saved to: {target_path}"

# Create the Gradio interface
iface = gr.Interface(
    fn=process_video,
    inputs=gr.Video(),  # Input for video upload
    outputs=[gr.Text()],  # Output for displaying the path to the saved video
    title="Vechile Speed Detection App",
    description="Upload a video to process it and save the output.",
)

# Launch the web application in debug mode
iface.launch(debug=True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://430da2b996f9ef16e9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Ultralytics Solutions: ✅ {'region': [(20, 400), (1080, 404), (1080, 360), (20, 360)], 'show_in': True, 'show_out': True, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': None, 'records': 5, 'model': 'yolo11n.pt', 'show': True}
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()



SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.


  0%|          | 0/2751 [00:00<?, ?it/s]